In [1]:
import requests
import json
import pandas as pd

In [2]:
import numpy as np

In [3]:
pd.options.display.max_columns = 999

In [4]:
fpl = 'https://fantasy.premierleague.com/api'
static_end = '/bootstrap-static/'
fixtures_end = '/fixtures/'
player_end = '/element-summary/'

### static information

In [5]:
r = requests.get(fpl+static_end)
static = json.loads(r.content)

In [6]:
players = pd.DataFrame(static['elements'])

In [7]:
events = pd.DataFrame(static['events'])

In [8]:
teams = pd.DataFrame(static['teams'])
teams['id'] = teams['id'].astype(str)

### fixtures

In [9]:
r = requests.get(fpl+fixtures_end)
fixtures = json.loads(r.content)

In [10]:
fixtures_df = pd.DataFrame(fixtures)
fixtures_df['event'].fillna(0,inplace=True)
fixtures_df['event'] = fixtures_df['event'].astype(str)
fixtures_df['team_a'] = fixtures_df['team_a'].astype(str)
fixtures_df['team_h'] = fixtures_df['team_h'].astype(str)

### player summary

In [11]:
player_id = '355/'
r = requests.get(fpl+player_end+player_id)
player_summary = json.loads(r.content)

In [12]:
player_summary.keys()

dict_keys(['fixtures', 'history', 'history_past'])

### gameweek live

In [13]:
live_end = '/event/1/live/'
r = requests.get(fpl+live_end)
gw = json.loads(r.content)

In [14]:
id_list = []
stat_list = []
for player in gw['elements']:
    id_list.append(player['id'])
    stat_list.append(player['stats'])
stats_df = pd.DataFrame(stat_list)
stats_df['id'] = id_list

### sandbox

In [15]:
teams_dict= teams.set_index('id')['short_name'].to_dict()

In [16]:
def get_teams_difficulty(gw=None):
    if not gw:
        gw = events.loc[events['is_current'],['id']].values[0][0]
    team_h = fixtures_df.loc[fixtures_df['event'] == gw, ['team_h', 'team_h_difficulty']].values
    team_a = fixtures_df.loc[fixtures_df['event'] == gw, ['team_a', 'team_a_difficulty']].values
    diff_list = np.concatenate((team_h, team_a))
    diff_df = pd.DataFrame(diff_list, columns=['team', 'difficulty'])
    diff_df.sort_values(by=['difficulty'],inplace=True)
    diff_df.reset_index(inplace=True)
    diff_df.drop(columns=['index'], inplace=True)
    return diff_df

In [17]:
fixtures_df.loc[((fixtures_df['team_a'] == '1') |
                (fixtures_df['team_h'] == '1')) &
                (fixtures_df['finished'] == False),
                ['team_h','team_h_difficulty','team_a','team_a_difficulty']].head(3).replace(teams_dict)

,team_h,team_h_difficulty,team_a,team_a_difficulty
40,ARS,3,MUN,4
47,EVE,4,ARS,2
57,ARS,3,TOT,4


In [18]:
def get_gameweek(event=1):
    url = f'https://fantasy.premierleague.com/api/event/{event}/live/'
    r = requests.get(url)
    raw = json.loads(r.content)
    id_list = []
    stat_list = []
    for player in raw['elements']:
        id_list.append(player['id'])
        stat_list.append(player['stats'])
    player_stats = pd.DataFrame(stat_list)
    player_stats['id'] = id_list
    player_stats['id'] = player_stats['id'].astype(str)
    return player_stats

In [19]:
finished_events= events.loc[events['finished']]['id'].tolist()

In [20]:
for event in finished_events:
    this_event = get_gameweek(event)
    

In [25]:
fixtures_df

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
0,2367552,0,False,False,15,None,0,False,None,6,NaN,12,NaN,[],2,2,93335
1,2367713,0,False,False,176,None,0,False,None,4,NaN,13,NaN,[],3,5,93496
2,2367538,1,True,True,1,2023-08-11T19:00:00Z,90,False,True,13,3.0,6,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",5,2,93321
3,2367540,1,True,True,2,2023-08-12T12:00:00Z,90,False,True,16,1.0,1,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,4,93322
4,2367539,1,True,True,3,2023-08-12T14:00:00Z,90,False,True,19,1.0,3,1.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,93323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2367913,38,False,False,376,2024-05-19T15:00:00Z,0,False,False,2,NaN,8,NaN,[],3,2,93696
376,2367914,38,False,False,377,2024-05-19T15:00:00Z,0,False,False,20,NaN,11,NaN,[],2,4,93697
377,2367915,38,False,False,378,2024-05-19T15:00:00Z,0,False,False,10,NaN,12,NaN,[],2,2,93698
378,2367916,38,False,False,379,2024-05-19T15:00:00Z,0,False,False,19,NaN,13,NaN,[],2,5,93699
